# 0. Setup Paths

In [ ]:
import os
from Preperation import Paths
CUSTOM_MODEL_NAME = 'raccoonModel'    # CHANGE THIS TO THE NAME OF THE MODEL YOU WANT TO EVALUATE

paths = Paths.paths

CHECKPOINT_PATH = os.path.join(paths.MODEL_PATH, CUSTOM_MODEL_NAME)

PIPELINE_CONFIG = os.path.join(paths.MODEL_PATH, CUSTOM_MODEL_NAME, 'pipeline.config')


# 1. Download Tensorflow Model and install TFOD

In [ ]:
if os.name=='nt':
    !pip install wget
    import wget

In [ ]:
if not os.path.exists(os.path.join(paths.APIMODEL_PATH, 'research', 'object_detection')):
    command = "git clone https://github.com/tensorflow/models {}".format(paths.APIMODEL_PATH)
    print(command)
    !{command}

In [ ]:
if os.name=='posix':  
    !apt-get install protobuf-compiler
    !cd Tensorflow/models/research && protoc object_detection/protos/*.proto --python_out=. && cp object_detection/packages/tf2/setup.py . && python -m pip install . 
    
if os.name=='nt':
    url="https://github.com/protocolbuffers/protobuf/releases/download/v3.15.6/protoc-3.15.6-win64.zip"
    wget.download(url)
    !move protoc-3.15.6-win64.zip {paths.PROTOC_PATH}
    !cd {paths.PROTOC_PATH} && tar -xf protoc-3.15.6-win64.zip
    os.environ['PATH'] += os.pathsep + os.path.abspath(os.path.join(paths.PROTOC_PATH, 'bin'))
    !cd Tensorflow/models/research && protoc object_detection/protos/*.proto --python_out=. && copy object_detection\\packages\\tf2\\setup.py setup.py && python setup.py build && python setup.py install
    !cd Tensorflow/models/research/slim && pip install -e . 

Verify that everything is setup correctly.  
The script should execute with 'OK (skipped=1)'

In [ ]:
VERIFICATION_SCRIPT = os.path.join(paths.APIMODEL_PATH, 'research', 'object_detection', 'builders', 'model_builder_tf2_test.py')
# Verify Installation
!python {VERIFICATION_SCRIPT}

In [ ]:
import object_detection

# 2. Verify that Model and Dataset Are Stored in their Respective Directories 

Before proceeding: Extract and copy the model to 'Tensorflow/workspace/models' if it is not already stored there! Pretrained models can be found in the folder ./pretrained_models. You have to unzip the model first.

In [ ]:
if os.path.exists(CHECKPOINT_PATH):
    print("Model available at: {}".format(CHECKPOINT_PATH))
else:
    print("Model not available at: {}".format(CHECKPOINT_PATH))

Before proceeding: Make sure that the directory 'Tensorflow/workspace/images/testset' exists. The folder 'testset' can be found in the dataset that we store on Google Drive.

In [ ]:
if len(os.listdir(paths.TESTSET_PATH)) > 0:
    print("Dataset available at: {}".format(paths.TESTSET_PATH))
else:
    print("Dataset not available at: {}".format(paths.TESTSET_PATH))

before evaluation can start make sure the paths are correct (when Training on Colab and Evaluation locally)

In [ ]:
import tensorflow as tf
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(PIPELINE_CONFIG, "r") as f:
    proto_str = f.read()
    text_format.Merge(proto_str, pipeline_config)

pipeline_config.train_input_reader.label_map_path= Paths.LABELMAP
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [Paths.TRAINSET_RECORD_PATH]
pipeline_config.eval_input_reader[0].label_map_path = Paths.LABELMAP
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [Paths.TESTSET_RECORD_PATH]

config_text = text_format.MessageToString(pipeline_config)

with tf.io.gfile.GFile(PIPELINE_CONFIG, "wb") as f:
    f.write(config_text)

# 3. Evaluate Model

In [ ]:
TRAINING_SCRIPT = os.path.join(paths.APIMODEL_PATH, 'research', 'object_detection', 'model_main_tf2.py')
command = "python {} --model_dir={} --pipeline_config_path={} --checkpoint_dir={}".format(TRAINING_SCRIPT, CHECKPOINT_PATH,PIPELINE_CONFIG, CHECKPOINT_PATH)
print(command.replace("\\", "/"))

In [ ]:
!{command}

# 4. Evaluate Model in Tensorboard

In [ ]:
!cd Tensorflow/workspace/models/{CUSTOM_MODEL_NAME}/train
print('Open http://www.localhost:6006/ in your browser')
!tensorboard --logdir=.

# 5. Load Trained Model From Checkpoint

In [ ]:
import os
import tensorflow as tf
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder
from object_detection.utils import config_util

In the following codeblock you have to change the term 'os.path.join(paths['CHECKPOINT_PATH'], 'ckpt-31')'.

Change 'ckpt-31' to the checkpoint ID of the pretrained model you want to use.

The checkpoint number can be found in ./Tensorflow/workspace/models/'MODEL NAME'/checkpoint/ckpt-XX.index

Whereas XX has to be replaced by the checkpoint ID you want to use.

In [ ]:
# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(PIPELINE_CONFIG)
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(tf.train.latest_checkpoint(CHECKPOINT_PATH)).expect_partial()

@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections

# 6. Detect from an Image

In [ ]:
import cv2 
import numpy as np
%matplotlib inline
from matplotlib import pyplot as plt

In [ ]:
category_index = label_map_util.create_category_index_from_labelmap(Paths.LABELMAP)

Run Image detection on an image

In [ ]:
directory = paths.TESTSET_PATH
detected_counts = {}

IMAGE_EXTENSIONS = ('.jpg', '.JPG', '.PNG', '.png')

for file in os.listdir(directory):
    if file.endswith(IMAGE_EXTENSIONS):
        filename = os.path.join(directory, file)
        print(filename)

        img = cv2.imread(filename)
        image_np = np.array(img)

        input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
        detections = detect_fn(input_tensor)

        num_detections = int(detections.pop('num_detections'))
        print(num_detections)
        detections = {key: value[0, :num_detections].numpy()
                    for key, value in detections.items()}
        detections['num_detections'] = num_detections

        # detection_classes should be ints.
        detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

        label_id_offset = 1
        image_np_with_detections = image_np.copy()


        threshhold = .30

        viz_utils.visualize_boxes_and_labels_on_image_array(
            image=image_np_with_detections,
            boxes=detections['detection_boxes'],
            classes=detections['detection_classes']+label_id_offset,
            scores=detections['detection_scores'],
            category_index=category_index,
            use_normalized_coordinates=True,
            max_boxes_to_draw=10,
            min_score_thresh=threshhold,
            agnostic_mode=False,
            line_thickness=4
        )

        plt.imshow(cv2.cvtColor(image_np_with_detections, cv2.COLOR_BGR2RGB))
        plt.show()